# Record Audio Using pyaudio module

In [ ]:
!pip install pyaudio==0.2.13

In [18]:
import pyaudio
p = pyaudio.PyAudio()
info = p.get_host_api_info_by_index(0)
numdevices = info.get('deviceCount')

for i in range(0, numdevices):
    if (p.get_device_info_by_host_api_device_index(0, i).get('maxInputChannels')) > 0:
        print("Input Device id ", i, " - ", p.get_device_info_by_host_api_device_index(0, i).get('name'))

Input Device id  0  -  HDA Intel MID: ALC888 Analog (hw:0,0)
Input Device id  2  -  HDA Intel MID: ALC888 Alt Analog (hw:0,2)
Input Device id  3  -  USB Audio Device: - (hw:1,0)
Input Device id  4  -  sysdefault
Input Device id  11  -  samplerate
Input Device id  12  -  speexrate
Input Device id  13  -  pulse
Input Device id  14  -  upmix
Input Device id  15  -  vdownmix
Input Device id  17  -  default


# Recoder Code

In [25]:
import os
import pyaudio
import wave
import ipywidgets as widgets
from IPython.display import display, Audio

class AudioRecorder:
    def __init__(self, file_path='recorded_audio.wav', channels=2, sample_rate=44100, chunk=1024):
        self.file_path = file_path
        self.file_dir = "logs"
        self.channels = channels
        self.increment = 0
        self.sample_rate = sample_rate
        self.chunk = chunk
        self.frames = []
        self.recording = False
        self.p = pyaudio.PyAudio()

        self.record_button = widgets.Button(description="Start Recording")
        self.stop_button = widgets.Button(description="Stop Recording", disabled=True)
        
        self.record_button.on_click(self.start_recording)
        self.stop_button.on_click(self.stop_recording)

    def callback(self, in_data, frame_count, time_info, status):
        if self.recording:
            self.frames.append(in_data)
        return None, pyaudio.paContinue

    def start_recording(self, _):
        self.frames = []
        self.recording = True
        self.record_button.disabled = True
        self.stop_button.disabled = False
        os.makedirs(self.file_dir, exist_ok=True)
#         print(self.p.get_default_input_device_info())

        self.stream = self.p.open(format=pyaudio.paInt16,
                channels=self.channels,
                rate=self.sample_rate,
                input=True,
                frames_per_buffer=self.chunk,
                stream_callback=self.callback)

        print("Recording...")
        self.stream.start_stream()

    def stop_recording(self, _):
        self.recording = False
        self.record_button.disabled = False
        self.stop_button.disabled = True
        self.stream.stop_stream()
        self.increment+=1
        self.stream.close()
        self.p.terminate()
        self.p = pyaudio.PyAudio()
        
        path_dir = os.path.join(self.file_dir,  self.file_path[:-4]+"_"+str(self.increment)+".wav")
        print("save file path : ", path_dir)
        
        print(len(self.frames))

        with wave.open(path_dir, 'wb') as wf:
            wf.setnchannels(self.channels)
            wf.setsampwidth(self.p.get_sample_size(pyaudio.paInt16))
            wf.setframerate(self.sample_rate)
            wf.writeframes(b''.join(self.frames))

        print(f"Audio recorded and saved to {self.file_path}")
        display(Audio(filename=self.file_path))

recorder = AudioRecorder()
widgets.VBox([recorder.record_button, recorder.stop_button])

Recording...
save file path :  logs/recorded_audio_1.wav
50
Audio recorded and saved to recorded_audio.wav


Recording...
save file path :  logs/recorded_audio_2.wav
72
Audio recorded and saved to recorded_audio.wav
